In [ ]:
!git init; git remote add origin https://github.com/EwaFengler/SmartElementData/; git pull origin master

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 3), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/EwaFengler/SmartElementData
 * branch            master     -> FETCH_HEAD
   3168995..8f211ae  master     -> origin/master
Updating 3168995..8f211ae
Fast-forward
 D/forward/16-05_18-53-57    | 76 --------------------------------------------
 D/forward/22-05_16-18-46    | 76 --------------------------------------------
 D/horizontal/16-05_18-46-42 | 76 --------------------------------------------
 D/horizontal/16-05_18-48-08 | 77 ---------------------------------------------
 D/vertical/16-05_18-56-39   | 76 --------------------------------------------
 5 files changed, 381 deletions(-)
 delete mode 100644 D/forward/16-05_18-53-57
 delete mode 10

In [ ]:
import pandas
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import numpy as np

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
# data_path = "MovesGravity/"
data_path = ""

DATA_SERIES = 3

SIZE = DATA_SERIES * 20

In [ ]:
def xd(path=data_path+"D/forward"):
    return [join(path, f) for f in listdir(path) if isfile(join(path, f))]

xd()

['D/forward/16-05_18-54-39',
 'D/forward/22-05_16-20-26',
 'D/forward/16-05_18-52-40',
 'D/forward/22-05_16-19-30',
 'D/forward/22-05_16-20-06',
 'D/forward/22-05_16-18-43',
 'D/forward/16-05_18-53-54',
 'D/forward/16-05_18-53-14',
 'D/forward/22-05_16-20-17',
 'D/forward/16-05_18-52-22',
 'D/forward/16-05_18-54-06',
 'D/forward/16-05_18-52-47',
 'D/forward/16-05_18-54-10',
 'D/forward/22-05_16-18-56',
 'D/forward/22-05_16-19-58',
 'D/forward/16-05_18-51-53',
 'D/forward/16-05_18-54-26',
 'D/forward/16-05_18-54-55',
 'D/forward/22-05_16-18-30',
 'D/forward/16-05_18-52-19',
 'D/forward/22-05_16-18-33',
 'D/forward/22-05_16-18-59',
 'D/forward/22-05_16-19-21',
 'D/forward/22-05_16-20-57',
 'D/forward/16-05_18-52-36',
 'D/forward/16-05_18-51-50',
 'D/forward/16-05_18-53-42',
 'D/forward/22-05_16-19-53',
 'D/forward/22-05_16-19-55',
 'D/forward/16-05_18-52-09',
 'D/forward/22-05_16-19-17',
 'D/forward/22-05_16-20-23',
 'D/forward/16-05_18-53-05',
 'D/forward/22-05_16-20-40',
 'D/forward/22

In [ ]:
def movesGenerator(path, y):
    def prepareMoves():
        files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
      
        for file in files:
            data = pandas.read_csv(file, sep=" ", names=["time", "x", "y", "z", "xg", "yg", "zg", "a", "p", "r"])

            del data['a']
            del data['p']
            del data['r']
            del data['z']
            del data['xg']
            del data['yg']

            data["time"] //= 1000000

            #16 obserwacji na 1s
            data["time"] //= 62.5 
            data = data.groupby('time').mean()

            #1 wymiar do sieci
            data = data[:19].to_numpy().flatten()

            yield data, y

    return prepareMoves

In [ ]:
def randomGenerator(path, step):
    def prepareRandom():
        files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
      
        for file in files:
            data = pandas.read_csv(file, sep=" ", names=["time", "x", "y", "z", "xg", "yg", "zg", "a", "p", "r"])

            del data['a']
            del data['p']
            del data['r']
            del data['z']
            del data['xg']
            del data['yg']

            data["time"] //= 1000000

            #16 obserwacji na 1s
            data["time"] //= 62.5 
            data = data.groupby('time').mean()

            i = 0
            while i + 20 < len(data):
                res = data[i:i+19].to_numpy().flatten()
                if len(res) == SIZE:
                    yield res, [0,0,0]
                # yield data[i:i+19]
                i += step
  
    return prepareRandom

In [ ]:
# from ipywidgets import interact, IntSlider

# @interact(k=IntSlider(min=0, max=276))
# def asdf(k):
#   i = 0
#   for d in randomGenerator("E/random")():
#     if(i == k):
#       d.plot()
#     i += 1
    

In [ ]:
ds_eh = tf.data.Dataset.from_generator(movesGenerator(data_path+"E/horizontal", [1,0,0]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_dh = tf.data.Dataset.from_generator(movesGenerator(data_path+"D/horizontal", [1,0,0]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_ev = tf.data.Dataset.from_generator(movesGenerator(data_path+"E/vertical", [0,1,0]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_dv = tf.data.Dataset.from_generator(movesGenerator(data_path+"D/vertical", [0,1,0]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_ef = tf.data.Dataset.from_generator(movesGenerator(data_path+"E/forward", [0,0,1]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_df = tf.data.Dataset.from_generator(movesGenerator(data_path+"D/forward", [0,0,1]), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_er = tf.data.Dataset.from_generator(randomGenerator(data_path+"E/random", 5), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)
ds_dr = tf.data.Dataset.from_generator(randomGenerator(data_path+"D/random", 5), (tf.float32, tf.int64), output_shapes=((SIZE,), (3,))) \
  .shuffle(100)

In [ ]:
#rozmiar: 1276

ds_all = ds_eh.concatenate(ds_dh) \
  .concatenate(ds_ev) \
  .concatenate(ds_dv) \
  .concatenate(ds_ef) \
  .concatenate(ds_df) \
  .concatenate(ds_er) \
  .concatenate(ds_dr) \
  .shuffle(2000) \
  .batch(32)

# #only D
# ds_all = ds_dh \
#   .concatenate(ds_dv) \
#   .concatenate(ds_df) \
#   .concatenate(ds_dr) \
#   .shuffle(2000) \
#   .batch(32)


# #only E
# ds_all = ds_eh \
#   .concatenate(ds_ev) \
#   .concatenate(ds_ef) \
#   .concatenate(ds_er) \
#   .shuffle(2000) \
#   .batch(32)

# ds_train = ds_eh.skip(15) \
#   .concatenate(ds_dh.skip(15)) \
#   .concatenate(ds_ev.skip(15)) \
#   .concatenate(ds_dv.skip(15)) \
#   .concatenate(ds_ef.skip(15)) \
#   .concatenate(ds_df.skip(15)) \
#   .concatenate(ds_er.skip(50)) \
#   .concatenate(ds_dr.skip(50)) \
#   .shuffle(2000) \
#   .batch(32)

# ds_test = ds_eh.take(15) \
#   .concatenate(ds_dh.take(15)) \
#   .concatenate(ds_ev.take(15)) \
#   .concatenate(ds_dv.take(15)) \
#   .concatenate(ds_ef.take(15)) \
#   .concatenate(ds_df.take(15)) \
#   .concatenate(ds_er.take(50)) \
#   .concatenate(ds_dr.take(50)) \
#   .shuffle(2000) \
#   .batch(32)

In [ ]:
for ds in [ds_eh, ds_ev, ds_ef, ds_er, ds_dh, ds_dv, ds_df, ds_dr]:
  print(len(list(ds)))

100
104
101
278
98
99
98
170


In [ ]:
model = tf.keras.Sequential([
      tf.keras.layers.Dense(100, 'relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64, 'relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(32, 'relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(3, 'sigmoid')
  ], name="model")

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])

# model.fit(ds_train, epochs=10, validation_data=ds_test, shuffle=True)
model.fit(ds_all, epochs=10, shuffle=True)

Epoch 1/10
33/33 [==============================] - 0s 2ms/step - loss: 0.7059 - binary_accuracy: 0.7630
Epoch 2/10
33/33 [==============================] - 0s 2ms/step - loss: 0.3305 - binary_accuracy: 0.8725
Epoch 3/10
33/33 [==============================] - 0s 2ms/step - loss: 0.2412 - binary_accuracy: 0.9068
Epoch 4/10
33/33 [==============================] - 0s 2ms/step - loss: 0.1772 - binary_accuracy: 0.9354
Epoch 5/10
33/33 [==============================] - 0s 2ms/step - loss: 0.1350 - binary_accuracy: 0.9536
Epoch 6/10
33/33 [==============================] - 0s 2ms/step - loss: 0.0986 - binary_accuracy: 0.9682
Epoch 7/10
33/33 [==============================] - 0s 2ms/step - loss: 0.0835 - binary_accuracy: 0.9701
Epoch 8/10
33/33 [==============================] - 0s 2ms/step - loss: 0.0721 - binary_accuracy: 0.9787
Epoch 9/10
33/33 [==============================] - 0s 2ms/step - loss: 0.0506 - binary_accuracy: 0.9866
Epoch 10/10
33/33 [==============================] - 0s

In [ ]:
model.predict(ds_df.take(1).batch(1))

array([[1.7044335e-04, 1.6818142e-04, 9.9928468e-01]], dtype=float32)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             multiple                  6100      
_________________________________________________________________
dropout_9 (Dropout)          multiple                  0         
_________________________________________________________________
dense_13 (Dense)             multiple                  6464      
_________________________________________________________________
dropout_10 (Dropout)         multiple                  0         
_________________________________________________________________
dense_14 (Dense)             multiple                  2080      
_________________________________________________________________
dropout_11 (Dropout)         multiple                  0         
_________________________________________________________________
dense_15 (Dense)             multiple                  99    

In [ ]:
model.save("MLP_model", include_optimizer=False)

INFO:tensorflow:Assets written to: MLP_model/assets


INFO:tensorflow:Assets written to: MLP_model/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

f = open('hopefully_final_model_xy_zgravity.tflite', 'wb')
f.write(tflite_model)
f.close()

STASH:


In [ ]:
assert False, "do not run rest of cells when using 'run all'"

AssertionError: ignored

In [ ]:
from ipywidgets import interact, IntSlider

H = [join("horizontal", f) for f in listdir("horizontal") if isfile(join("horizontal", f))]

@interact(i=IntSlider(min=0, max=len(H)-1))
def show(i):
    data = pandas.read_csv(H[i], sep=" ", names=["time", "x", "y", "z", "a", "p", "r"])
    
    data["time"] //= 1000000
    
    #16 obserwacji na 1s
    data["time"] //= 62.5 
    data = data.groupby('time').mean()
    
    print(data[:19].to_numpy().flatten().dtype)
#     plt.figure()
#     data['x'].plot()
#     data['y'].plot()
#     data['z'].plot()


In [ ]:
def approxByKPoints(x, k):
    points = [int(round(p)) for p in np.linspace(0, len(x), k+1)]
    
    res = [np.mean(x[points[i]:points[i+1]]) for i in range(k)]      
        
    return res

print(approxByKPoints([1,2,3,4,5,4,3,2,1,2], 3))